# Install Required Dependencies

In [ ]:
!pip install vitaldb

In [ ]:
import datetime
import os
import random
import time

from timeit import default_timer as timer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

import vitaldb

# Set Random Seed for Reproducibility

In [ ]:
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.use_deterministic_algorithms(True)
os.environ['PYTHONASHSEED'] = str(seed)

# Set Up Local Data Caches

Since the VitalDB data is static, local copies are stored and reused to avoid expensive downloads and to speed up data processing.

The default directory defined below is already in the project `.gitignore` file. If later modified, it should also be added to the project `.gitignore`.

In [ ]:
VITALDB_CACHE = './vitaldb_cache'
VITAL_ALL = 'vital_all'
VITAL_MINI = 'vital_mini'
VITAL_METADATA = 'metadata'

In [ ]:
!mkdir -p $VITALDB_CACHE
!mkdir -p $VITALDB_CACHE/$VITAL_ALL
!mkdir -p $VITALDB_CACHE/$VITAL_MINI
!mkdir -p $VITALDB_CACHE/$VITAL_METADATA
!ls -l $VITALDB_CACHE

# OSFS Bulk Data Download

**This step is not required, but will significantly speed up downstream processing and avoid a high volume of API requests to the VitalDB web site.**

The cache population code checks if OSFS bulk download data of VitalDB vital files is locally available.

- Manually downloaded the OSF Store archives from the following site: https://osf.io/dtc45/
    - `Vital Files 0001-2000`
    - `Vital Files 2001-4000`
    - `Vital Files 4001-6388`
- Once the `OSF Storage (United States)` link is clicked a `Download as zip` link will appear.
- Once downloaded, extract each of the 3 zip archives.
- Move all files from each of the unzip directories into the `${VITALDB_CACHE}/${VITAL_ALL}` directory.

In [ ]:
# Returns the Pandas DataFrame for the specified dataset.
#   One of 'cases', 'labs', or 'trks'
# If the file exists locally, create and return the DataFrame.
# Else, download and cache the csv first, then return the DataFrame.
def vitaldb_dataframe_loader(dataset_name):
    if dataset_name not in ['cases', 'labs', 'trks']:
        raise ValueError(f'Invalid dataset name: {dataset_name}')
    file_path = f'{VITALDB_CACHE}/{VITAL_METADATA}/{dataset_name}.csv'
    if os.path.isfile(file_path):
        print(f'{dataset_name}.csv exists locally.')
        df = pd.read_csv(file_path)
        return df
    else:
        print(f'downloading {dataset_name} and storing in the local cache for future reuse.')
        df = pd.read_csv(f'https://api.vitaldb.net/{dataset_name}')
        df.to_csv(file_path, index=False)
        return df

# Cases

In [ ]:
cases = vitaldb_dataframe_loader('cases')
cases = cases.set_index('caseid')
cases.shape

In [ ]:
cases.index.nunique()

In [ ]:
cases.head()

In [ ]:
cases['sex'].value_counts()

# Tracks

In [ ]:
trks = vitaldb_dataframe_loader('trks')
trks = trks.set_index('caseid')
trks.shape

In [ ]:
trks.index.nunique()

In [ ]:
trks.groupby('caseid')[['tid']].count().plot();

In [ ]:
trks.groupby('caseid')[['tid']].count().hist();

In [ ]:
trks.groupby('tname').count().sort_values(by='tid', ascending=False)

## Parameters of Interest

### Hemodynamic Parameters Reference
https://vitaldb.net/dataset/?query=overview#h.f7d712ycdpk2

**Solar8000/ART_MBP**

mean blood pressure

Parameter, Description, Type/Hz, Unit

Solar8000/ART_MBP, Mean arterial pressure, N, mmHg

In [ ]:
trks[trks['tname'].str.contains('Solar8000/ART_MBP')].shape

**SNUADC/ART**

arterial blood pressure waveform

Parameter, Description, Type/Hz, Unit

SNUADC/ART, Arterial pressure wave, W/500, mmHg

In [ ]:
trks[trks['tname'].str.contains('SNUADC/ART')].shape

**SNUADC/ECG_II**

electrocardiogram waveform

Parameter, Description, Type/Hz, Unit

SNUADC/ECG_II, ECG lead II wave, W/500, mV

In [ ]:
trks[trks['tname'].str.contains('SNUADC/ECG_II')].shape

**BIS/EEG1_WAV**

electroencephalogram waveform

Parameter, Description, Type/Hz, Unit

BIS/EEG1_WAV, EEG wave from channel 1, W/128, uV

In [ ]:
trks[trks['tname'].str.contains('BIS/EEG1_WAV')].shape

# Cases of Interest

These are the subset of case ids for which modelling and analysis will be performed based upon inclusion criteria and waveform data availability.

In [ ]:
TRACK_NAMES = ['SNUADC/ART', 'SNUADC/ECG_II', 'BIS/EEG1_WAV']

In [ ]:
# As in the paper, select cases which meet the following criteria:
#
# For patients, the inclusion criteria were as follows:
# (1) adults (age >= 18)
# (2) administered general anaesthesia
# (3) undergone non-cardiac surgery. 
#
# For waveform data, the inclusion criteria were as follows:
# (1) no missing monitoring for ABP, ECG, and EEG waveforms
# (2) no cases containing false events or non-events due to poor signal quality
#     (checked in second stage of data preprocessing)

# adult
inclusion_1 = cases.loc[cases['age'] >= 18].index
print(f'{len(cases)-len(inclusion_1)} cases excluded, {len(inclusion_1)} remaining due to age criteria')

# general anesthesia
inclusion_2 = cases.loc[cases['ane_type'] == 'General'].index
print(f'{len(cases)-len(inclusion_2)} cases excluded, {len(inclusion_2)} remaining due to anesthesia criteria')

# non-cardiac surgery
inclusion_3 = cases.loc[
    ~cases['opname'].str.contains("cardiac", case=False)
    & ~cases['opname'].str.contains("aneurysmal", case=False)
].index
print(f'{len(cases)-len(inclusion_3)} cases excluded, {len(inclusion_3)} remaining due to non-cardiac surgery criteria')

# ABP, ECG, EEG waveforms
TRACK_NAMES = ['SNUADC/ART', 'SNUADC/ECG_II', 'BIS/EEG1_WAV']
inclusion_4 = trks.loc[trks['tname'].isin(TRACK_NAMES)].index.value_counts()
inclusion_4 = inclusion_4[inclusion_4 == len(TRACK_NAMES)].index
print(f'{len(cases)-len(inclusion_4)} cases excluded, {len(inclusion_4)} remaining due to missing waveform data')

cases_of_interest_idx = inclusion_1 \
    .intersection(inclusion_2) \
    .intersection(inclusion_3) \
    .intersection(inclusion_4)

cases_of_interest = cases.loc[cases_of_interest_idx]

print()
print(f'{cases_of_interest_idx.shape[0]} out of {cases.shape[0]} total cases remaining after exclusions applied')

In [ ]:
cases_of_interest.head(n=5)

# Tracks of Interest

These are the subset of tracks (waveforms) for the cases of interest identified above.

In [ ]:
# A single case maps to one or more waveform tracks. Select only the tracks required for analysis.
trks_of_interest = trks.loc[cases_of_interest_idx][trks.loc[cases_of_interest_idx]['tname'].isin(TRACK_NAMES)]
trks_of_interest.shape

In [ ]:
trks_of_interest.head(n=5)

In [ ]:
trks_of_interest_idx = trks_of_interest.set_index('tid').index
trks_of_interest_idx.shape

## Build Tracks Cache for Local Processing

Tracks data are large and therefore expensive to download every time used.
By default, the vital file format stores all tracks for each case internally. Since only certain tracks per case are required, each vital file can be further truncated to only store the tracks for needed waveforms.

In [ ]:
# Maximum number of cases of interest for which to download data.
# Set to a small value for demo purposes, else set to None to disable and download all.
MAX_CASES = None
#MAX_CASES = 10

In [ ]:
# Ensure the full vital file dataset is available for cases of interest.
count_downloaded = 0
count_present = 0

#for i, idx in enumerate(cases.index):
for i, idx in enumerate(cases_of_interest_idx):
    if MAX_CASES and i >= MAX_CASES:
        break

    full_path = f'{VITALDB_CACHE}/{VITAL_ALL}/{idx:04d}.vital'
    if not os.path.isfile(full_path):
        print(f'Missing vital file: {full_path}')
        # Download and save the file.
        vf = vitaldb.VitalFile(idx)
        vf.to_vital(full_path)
        count_downloaded += 1
    else:
        count_present += 1

print()
print(f'Count of cases of interest:           {cases_of_interest_idx.shape[0]}')
print(f'Count of vital files downloaded:      {count_downloaded}')
print(f'Count of vital files already present: {count_present}')

In [ ]:
# Convert vital files to "mini" versions including only the subset of tracks based on TRACK_NAMES defined above.
# Only perform conversion for the cases of interest.
# NOTE: If this cell is interrupted, it can be restarted and will continue where it left off.
count_minified = 0
count_present = 0

for i, idx in enumerate(cases_of_interest_idx):
    if MAX_CASES and i >= MAX_CASES:
        break
    
    full_path = f'{VITALDB_CACHE}/{VITAL_ALL}/{idx:04d}.vital'
    mini_path = f'{VITALDB_CACHE}/{VITAL_MINI}/{idx:04d}_mini.vital'
    if not os.path.isfile(mini_path):
        print(f'Creating mini vital file: {idx}')
        vf = vitaldb.VitalFile(full_path, TRACK_NAMES)
        vf.to_vital(mini_path)
        count_minified += 1
    else:
        count_present += 1

print()
print(f'Count of cases of interest:           {cases_of_interest_idx.shape[0]}')
print(f'Count of vital files minified:        {count_minified}')
print(f'Count of vital files already present: {count_present}')

# Track Plotting Examples

These examples show multiple ways of accessing the same track data.

In [ ]:
tmp_vf_path = f'{VITALDB_CACHE}/{VITAL_MINI}/0001_mini.vital'

In [ ]:
tmp_vf = vitaldb.VitalFile(tmp_vf_path)
tmp_vf.get_track_names()

**`vitaldb.VitalFile.get_track_samples()`**

In [ ]:
tmp_art_00 = tmp_vf.get_track_samples(TRACK_NAMES[0], 1/100)
tmp_art_00.shape

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(tmp_art_00)
plt.show()

**`vitaldb.VitalFile.to_numpy()`**

In [ ]:
tmp_art_01 = tmp_vf.to_numpy(TRACK_NAMES[0], 1/100)
tmp_art_01.shape

In [ ]:
type(tmp_art_01)

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(tmp_art_01)
plt.show()

**`vitaldb.VitalFile.get_samples()`**

This option allows returning timestamps as well.

In [ ]:
tmp_art_02 = tmp_vf.get_samples(TRACK_NAMES[0], 1/100, return_timestamp=True)
tmp_art_02

In [ ]:
tmp_art_02[0][0][-1] - tmp_art_02[0][0][0]

In [ ]:
type(tmp_art_02[0][0])

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(tmp_art_02[0][1])
plt.show()

**`vitaldb.vital_recs()`**

In [ ]:
tmp_art_03 = vitaldb.vital_recs(tmp_vf_path, TRACK_NAMES[0], 1/100)
tmp_art_03.shape

In [ ]:
type(tmp_art_03)

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(tmp_art_03)
plt.show()

**`vitaldb.dataset.load_trk()`**

NOTE: This downloads a track based on raw id. Should not be needed, but showing how to do it.

In [ ]:
SHOW_TRAK_DOWNLOAD = False

if SHOW_TRAK_DOWNLOAD:
    case_0001_trk_art_id = '724cdd7184d7886b8f7de091c5b135bd01949959'
    tmp_art_04 = vitaldb.dataset.load_trk(case_0001_trk_art_id, 1/100)
    print(tmp_art_04.shape)
    
    plt.figure(figsize=(20, 5))
    plt.plot(tmp_art_04)
    plt.show()
else:
    print('Manual track download example skipped.')

# One Minute ABP Extraction

In [ ]:
tmp_vf.dtstart

In [ ]:
start = datetime.datetime.fromtimestamp(tmp_vf.dtstart).isoformat()
start

In [ ]:
end = datetime.datetime.fromtimestamp(tmp_vf.dtend).isoformat()
end

In [ ]:
datetime.datetime.fromtimestamp(tmp_vf.dtend) - datetime.datetime.fromtimestamp(tmp_vf.dtstart)

In [ ]:
d = tmp_vf.get_dt(2024, 4, 1)
d

In [ ]:
datetime.datetime.fromtimestamp(d).isoformat()

In [ ]:
dd = tmp_vf.get_samples(TRACK_NAMES, 1)
type(dd)

In [ ]:
tmp_art_00 = tmp_vf.get_track_samples(TRACK_NAMES[0], 1)
type(tmp_art_00)

In [ ]:
z = tmp_art_00[tmp_art_00 > 0]
z.shape

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(z)
plt.show()

In [ ]:
z = tmp_art_00[1000:10000].copy()
z = z[z > 0]

plt.figure(figsize=(20, 5))
plt.plot(z)
plt.show()

In [ ]:
for caseid in cases_of_interest_idx[:1]:
    print(caseid)
    tmp_vf_path = f'{VITALDB_CACHE}/{VITAL_MINI}/{caseid:04d}_mini.vital'
    print(tmp_vf_path)
    tmp_vf = vitaldb.VitalFile(tmp_vf_path)
    arts = tmp_vf.to_numpy(TRACK_NAMES[0], 1/100)
    print(arts.ravel().shape)

In [ ]:
# Generate hypotensive events
# Hypotensive events are defined as a 1-minute interval with sustained ABP of less than 65 mmHg
# Note: Hypotensive events should be at least 20 minutes apart to minimize potential residual effects from previous events
# TODO: Implement hypotension event generation function
# TODO: Generate hypotension events

# Generate hypotension non-events
# To sample non-events, 30-minute segments where the ABP was above 75 mmHG were selected, and then
# three one-minute samples of each waveform were obtained from the middle of the segment
# TODO: Implement hypotension non-event generation function
# TODO: Generate hypotension non-events

Adapted from: https://github.com/vitaldb/examples/blob/master/hypotension_art.ipynb

In [ ]:
TRACK_NAMES

In [ ]:
tmp_vf.get_track_samples(TRACK_NAMES[0], 1/500).shape

In [ ]:
tmp_vf.get_track_samples(TRACK_NAMES[1], 1/500).shape

In [ ]:
tmp_vf.get_track_samples(TRACK_NAMES[2], 1/128).shape

In [ ]:
tmp_vf.get_track_samples(TRACK_NAMES[2], 1/128).shape

In [ ]:
def extract_segments(cases_of_interest_idx, min_before_event=3, debug=False):
    # Sampling rate for ABP and ECG, Hz. These rates should be the same. Default = 500
    ABP_ECG_SRATE_HZ = 500

    # Sampling rate for EEG. Default = 128
    EEG_SRATE_HZ = 128

    # Length of feature segment, seconds.
    FEATURE_LENGTH_SEC = 60
    # Look ahead to predict hypotension, seconds.
    MIDDLE_LENGTH_SEC  = 60 * min_before_event
    # Length of label segment, seconds.
    LABEL_LENGTH_SEC   = 60

    # Length to move down the ABP track for starting a new analysis segment, seconds.
    NEW_SEGMENT_OFFSET_SEC = 10

    # Final dataset for training and testing the model.
    # inputs with shape of (segments, timepoints)
    x_abp = []
    x_ecg = []
    x_eeg = []

    # output with shape of (segments)
    y = []

    # validity of each segement; only valid segments are used for model building
    valid_mask = []

    # caseid of each segment
    case_id_per_segment = []

    # Process each case and extract segments. For each segment identify presence of an event in the label zone.
    time_start = timer()
    
    count_cases = len(cases_of_interest_idx)

    for case_count, caseid in enumerate(cases_of_interest_idx):
        if debug:
            print(f'Loading case: {caseid:04d}, ({case_count + 1} of {count_cases})')

        # read the arterial waveform
        tmp_vf_path = f'{VITALDB_CACHE}/{VITAL_MINI}/{caseid:04d}_mini.vital'
        tmp_vf = vitaldb.VitalFile(tmp_vf_path)

        abp = tmp_vf.to_numpy(TRACK_NAMES[0], 1/ABP_ECG_SRATE_HZ).ravel()
        ecg = tmp_vf.to_numpy(TRACK_NAMES[1], 1/ABP_ECG_SRATE_HZ).ravel()
        eeg = tmp_vf.to_numpy(TRACK_NAMES[2], 1/EEG_SRATE_HZ).ravel()

        # EEG - Different sample rate, process alone
        if debug:
            print(f'Length of {TRACK_NAMES[2]}:     {eeg.shape[0]}')

        print_first_segment = True

        last_sample_start_index = len(eeg) - EEG_SRATE_HZ * (FEATURE_LENGTH_SEC + MIDDLE_LENGTH_SEC + LABEL_LENGTH_SEC)
        sample_index_offset = NEW_SEGMENT_OFFSET_SEC * EEG_SRATE_HZ

        for i in range(0, last_sample_start_index, sample_index_offset):
            segx_start = i
            segx_end   = i + EEG_SRATE_HZ * FEATURE_LENGTH_SEC
            segx = eeg[segx_start:segx_end]

            if debug and print_first_segment:
                print(f'  Feature Segment Length:   {segx.shape[0]} pts, {segx.shape[0] / EEG_SRATE_HZ} sec')
                print_first_segment = False

            # handle eeg, only care about extracting data from the same time interval used for abp
            x_eeg.append(eeg[segx_start:segx_end])    

        # ABP and ECG - Shared sample rate, process together    
        if debug:
            print(f'Length of {TRACK_NAMES[0]}:       {abp.shape[0]}')
            print(f'Length of {TRACK_NAMES[1]}:    {ecg.shape[0]}')

        segment_count = 0
        segment_valid = 0
        segment_event = 0
        print_first_segment = True

        last_sample_start_index = len(abp) - ABP_ECG_SRATE_HZ * (FEATURE_LENGTH_SEC + MIDDLE_LENGTH_SEC + LABEL_LENGTH_SEC)
        sample_index_offset = NEW_SEGMENT_OFFSET_SEC * ABP_ECG_SRATE_HZ

        for i in range(0, last_sample_start_index, sample_index_offset):
            segment_count += 1

            segx_start = i
            segx_end   = i + ABP_ECG_SRATE_HZ * FEATURE_LENGTH_SEC
            segx = abp[segx_start:segx_end]

            segy_start = i + ABP_ECG_SRATE_HZ * (FEATURE_LENGTH_SEC + MIDDLE_LENGTH_SEC)
            segy_end   = i + ABP_ECG_SRATE_HZ * (FEATURE_LENGTH_SEC + MIDDLE_LENGTH_SEC + LABEL_LENGTH_SEC)
            segy = abp[segy_start:segy_end]

            if debug and print_first_segment:
                print(f'  Feature Segment Length:   {segx.shape[0]} pts, {segx.shape[0] / ABP_ECG_SRATE_HZ} sec')
                print(f'  Middle Segment Length:    {segy_start - segx_end} pts, {(segy_start - segx_end) / ABP_ECG_SRATE_HZ} sec')
                print(f'  Label Segment Length:     {segy.shape[0]} pts, {segy.shape[0] / ABP_ECG_SRATE_HZ} sec')
                print_first_segment = False

            # check the validity of this segment
            valid = True
            if np.isnan(segx).mean() > 0.1:
                valid = False
            elif np.isnan(segy).mean() > 0.1:
                valid = False
            elif (segx > 200).any():
                valid = False
            elif (segy > 200).any():
                valid = False
            elif (segx < 30).any():
                valid = False
            elif (segy < 30).any():
                valid = False
            elif np.max(segx) - np.min(segx) < 30:
                valid = False
            elif np.max(segy) - np.min(segy) < 30:
                valid = False
            elif (np.abs(np.diff(segx)) > 30).any():  # abrupt change -> noise
                valid = False
            elif (np.abs(np.diff(segy)) > 30).any():  # abrupt change -> noise
                valid = False

            # 2 sec moving avg
            n = 2 * ABP_ECG_SRATE_HZ  
            segy = np.nancumsum(segy, dtype=np.float32)
            segy[n:] = segy[n:] - segy[:-n]
            segy = segy[n - 1:] / n

            # forward filling - do this per case to avoid massive resource utilization at the end.
            segx = pd.DataFrame(segx).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0)[0].values

            # Identify IOH event as < 65mm HG
            evt = np.nanmax(segy) < 65
            x_abp.append(segx)
            y.append(evt)
            valid_mask.append(valid)
            case_id_per_segment.append(caseid)

            # handle ecg, only care about extracting the same segment used for abp.
            # data is already time aligned and has same sample rate.
            x_ecg.append(ecg[segx_start:segx_end])

            if valid:
                segment_valid += 1
                if evt:
                    segment_event += 1

        if debug:
            print(f'Total Segments Evaluated:   {segment_count}')
            segment_valid_percent = 0 if segment_count == 0 else 100 * segment_valid / segment_count 
            print(f'  Segments Valid:           {segment_valid}, {segment_valid_percent:.1f}%')
            segment_event_percent = 0 if segment_valid == 0 else 100 * segment_event / segment_valid
            print(f'  Segments with Event:      {segment_event}, {segment_event_percent:.1f}%')
            time_delta = np.round(timer() - time_start, 3)
            print(f'Total Processing Time:      {time_delta:.4f} sec')
            print()

    # final caseids
    caseids = np.unique(case_id_per_segment)

    # convert lists to numpy array
    x_abp = np.array(x_abp)
    y = np.array(y) 
    valid_mask = np.array(valid_mask)
    case_id_per_segment = np.array(case_id_per_segment)

    # forward filling
    #x_abp = pd.DataFrame(x_abp).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
    #x_abp = pd.DataFrame(x_abp).values

    # ABP normalization
    x_abp -= 65
    x_abp /= 65

    # add axis for CNN; i.e. (# samples, feature segment length) -> (# samples, feature segment length, 1)
    x_abp = x_abp[..., None]

    # handle ecg
    x_ecg = np.array(x_ecg)
    x_ecg = x_ecg[..., None]

    # handle eeg
    x_eeg = np.array(x_eeg)
    x_eeg = x_eeg[..., None]

    # total processing time
    time_end = timer()
    time_delta = np.round(time_end - time_start, 3)

    if debug:
        print('OVERALL SUMMARY')
        print(f'Total Processing Time:      {time_delta:.4f} sec')
        print(f'Total Cases Processed:      {caseids.shape[0]}')
        print(f'Total Segments Evaluated:   {x_abp.shape[0]}')

        segment_valid_count = np.sum(valid_mask)
        segment_valid_percent = 0 if x_abp.shape[0] == 0 else 100 * segment_valid_count / x_abp.shape[0] 
        print(f'  Segments Valid:           {segment_valid_count}, {segment_valid_percent:.1f}%')
        segment_event_count = np.sum(y & valid_mask)
        segment_event_percent = 0 if y.shape[0] == 0 else 100 * segment_event_count / y.shape[0]
        print(f'  Segments with Event:      {segment_event_count}, {segment_event_percent:.1f}%')


        print(f'Valid Samples Generated:    {(100 * np.mean(valid_mask)):.1f}%')
        print()
        print(f'Valid Mask Shape:           {valid_mask.shape}')
        print(f'X_ABP Shape:                {x_abp.shape}')
        print(f'X_ECG Shape:                {x_ecg.shape}')
        print(f'X_EEG Shape:                {x_eeg.shape}')
        print(f'Y Shape:                    {y.shape}')
        print(f'CIPS Shape:                 {case_id_per_segment.shape}')
    
    return (x_abp, x_ecg, x_eeg, y, valid_mask, case_id_per_segment)

In [ ]:
CASE_COUNT = 2

In [ ]:
x_abp, x_ecg, x_eeg, y, valid_mask, case_id_per_segment = extract_segments(
    cases_of_interest_idx[:CASE_COUNT], min_before_event=3, debug=True)

In [ ]:
x_abp, x_ecg, x_eeg, y, valid_mask = extract_segments(
    cases_of_interest_idx[:CASE_COUNT], min_before_event=5, debug=True)

In [ ]:
x_abp, x_ecg, x_eeg, y, valid_mask = extract_segments(
    cases_of_interest_idx[:CASE_COUNT], min_before_event=10, debug=True)

In [ ]:
x_abp, x_ecg, x_eeg, y, valid_mask = extract_segments(
    cases_of_interest_idx[:CASE_COUNT], min_before_event=15, debug=True)

In [ ]:
np.sum(y & valid_mask)

In [ ]:
plt.plot(valid_mask);

In [ ]:
valid_mask[190]

In [ ]:
plt.plot(x_abp[190].ravel());

In [ ]:
plt.plot(x_ecg[190].ravel());

In [ ]:
plt.plot(x_eeg[190].ravel());

In [ ]:
plt.plot(y);

In [ ]:
xx = pd.DataFrame(x_abp.squeeze())
xx.info()